In [1]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [3]:
def simple_get(url):
    
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        print('Loading page: ' + url)
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [4]:
raw_html = simple_get('https://www.omnical.co/electrical-parts-catalog')

Loading page: https://www.omnical.co/electrical-parts-catalog


In [5]:
ALP = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V' ,'W', 'X', 'Z']
strings = list()
urls = list()
brands = list()
html = BeautifulSoup(raw_html, 'html.parser')
# with open('persons.csv', 'w', newline='') as csvfile:
#     filewriter = csv.writer(csvfile, delimiter=',',
#                             quotechar='|', quoting=csv.QUOTE_MINIMAL)
for i, alp in zip(range(len(ALP)), ALP):
    strings.append(html.find('div', attrs={'id':'brand-'+alp}))    
    for a in strings[i].find_all('a', href=True):    
#             filewriter.writerow([a['href'], a.text])
        urls.append(a['href'])
        brands.append(a.text)    

In [6]:
def clean_html(url_html):
    url_text = BeautifulSoup(url_html, 'html.parser')
    view = url_text.find('div', attrs={'id':'products-results-view'})
    return view

In [17]:
def get_array(view):
    global next_url
    next_url = view.find('a', attrs={'rel':'next'})['href']
    next_page = view.find('a', attrs={'rel':'next'})['page']
    next_page = int(next_page) - 1
    with open(r'products.csv', 'a', newline='') as f:
        writer = csv.writer(f)        
        views  = view.findAll('div', attrs={'class':'tradeproduct product-row'})
        print('writing page '+ str(next_page ))
        for i, vw in zip(range(len(views)), views):
            brand = brands[current_brand]
            product =  vw.find('div', attrs={'class':'tradeproduct-title'}).text
            
            desc = vw.find('div', attrs={'class':'tradeproduct-generated-description-search'}).text
            
            if len(product.split(" ")) >= 4:
                product_title = product.split(" ")[3]
                product_group = product.split(" ")[1]
                product_code = product.split(" ")[2]
            else:
                product_title = product
                product_group = product
                product_code = product
            
            price = vw.find('span', attrs={'itemprop':'pricecurrency'}).text +" " + vw.find('span', attrs={'itemprop':'price'}).text
            product_detail_image = vw.find('img')['src']
            writer.writerow([brand, product_title, product_group, product_code, price, product_detail_image,"", "", product,"", desc])




In [45]:
current_brand = 92
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(30)
driver.get(urls[current_brand])

In [46]:
for x in range(2050):
    global driver
    try:      
        wait = WebDriverWait(driver, 20)
        element = wait.until(EC.visibility_of_element_located((By.ID, 'products-results-view')))
        if element.is_displayed():
            get_array(clean_html(driver.page_source))
            elem = driver.find_element_by_xpath("//div[@class='pages']/ul[1]/li/a[@rel='next']")
            elem.click()            
            sleep(3)
        else:
            print("Element not found")
    except TimeoutException:
        print("Loading took too much time!")


writing page 1
writing page 2
Loading took too much time!
Loading took too much time!
Loading took too much time!
writing page 3
writing page 4
writing page 5
writing page 6
writing page 7
writing page 8
writing page 9
writing page 10
writing page 11
writing page 12
writing page 13
writing page 14
writing page 15
writing page 16
writing page 17
writing page 18
writing page 19
writing page 20
writing page 21
writing page 22
writing page 23
writing page 24
writing page 25
writing page 26
writing page 27
writing page 28
writing page 29
writing page 30
writing page 31
writing page 32
writing page 33
writing page 34
writing page 35
writing page 36
writing page 37
writing page 38
writing page 39
writing page 40
writing page 41
writing page 42
writing page 43
writing page 44
writing page 45
writing page 46
writing page 47
writing page 48
writing page 49
writing page 50
writing page 51
writing page 52
writing page 53
writing page 54
writing page 55
writing page 56
writing page 57
writing page 

AttributeError: 'NoneType' object has no attribute 'is_displayed'

In [44]:
driver.quit()